# Database programming with MySQL

### 1. Implement Inventory Management in Python with MySQL

a) Inventory management, a critical element of the supply chain, is the tracking of inventory from manufacturers to warehouses and from these facilities to a point of sale. The goal of inventory management is to have the right products in the right place at the right time.

b) The required Database is Inventory, and the required Tables are Purchases, Sales and Inventory

c) Note: Apply your thoughts to demonstrate the DB Operation in Python.


In [1]:
import mysql.connector
conn=mysql.connector.connect(user='root',
                            password='123',
                            host='127.0.0.1')

In [2]:
# Create a database
cursor = conn.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS Inventory")


In [4]:
# Create a database
cursor = conn.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS Inventory")
# Connect to the Inventory database
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="123",
    database="Inventory"
)


cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Purchases (
        purchase_id INT AUTO_INCREMENT PRIMARY KEY,
        product_name VARCHAR(255),
        quantity INT,
        purchase_date DATE
    )
""")



In [8]:

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Sales (
      Sale_id INT AUTO_INCREMENT PRIMARY KEY,
      Product_name VARCHAR(255),
      quantity INT,
      sale_date DATE
    )
""")
# Create Sales table
cursor.execute("""
   CREATE TABLE IF NOT EXISTS Sales (
       sale_id INT AUTO_INCREMENT PRIMARY KEY,
       product_name VARCHAR(255),
       quantity INT,
       sale_date DATE
   )
""")


ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'AUTO INCREMENT PRIMARY KEY,
      Product_name VARCHAR(255),
      quantity INT,' at line 2

In [6]:
# Creatong Sales table
cursor.execute("""
   CREATE TABLE IF NOT EXISTS Sales (
       sale_id INT AUTO_INCREMENT PRIMARY KEY,
       product_name VARCHAR(255),
       quantity INT,
       sale_date DATE
   )
""")


In [9]:
# Creating Inventory table
cursor.execute("""
   CREATE TABLE IF NOT EXISTS Inventory (
       product_name VARCHAR(255) PRIMARY KEY,
       quantity INT
   )
""")


In [10]:
cursor.execute("""SELECT * FROM Sales""")

### 2. Customer Order Processing

Scenario: You are building a customer order processing system for an e-commerce system needs to interact with a MySQL database to store customer orders, products, and order details,

 1. Design a MySQL database schema for the order processing system, Including tables for customers, products, and orders.

 2. Write a Python program that connects to the database and allows customers to place new orders.

 3. Implement a feature that calculates the total cost of an order and updates product quantities in the database.

 4. How would you handle cases where a product is no longer available when a customer places an order? 5. Develop a function to generate order reports for the company's finance department.


In [ ]:
# 1.Design a MySQL database schema for the order processing system, Including tables for customers, products, and orders.

'''
Tables:

Customers Table:
customer_id (Primary Key)
first_name
last_name
emailaddress
phone_number

Products Table:
product_id (PrimaryKey)
product_name
price
quantity_available

Orders Table:
order_id (Primary Key)
customer_id (Foreign Key references Customers)
order_datestatus (e.g., Processing, Shipped, Delivered)

Order_Details Table:
order_detail_id (Primary Key)
order_id (Foreign Key references Orders)
product_id (Foreign Key references Products)
quantity_ordered
total_price
'''

In [ ]:
import mysql.connector
conn=mysql.connector.connect(user='root',
                            password='jeffmessi123',
                            host='127.0.0.1')
host = "127.0.0.1"
username = "root"
password = "123"
database_name = "order_processing"

# Establish a connection to MySQL server
conn = mysql.connector.connect(
    host=host,
    user=username,
    password=password,
)
cursor = conn.cursor()

# Create the database if it doesn't exist
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")

# Switch to the newly created database
cursor.execute(f"USE {database_name}")

# Define SQL commands to create tables
create_customers_table = """
CREATE TABLE IF NOT EXISTS Customers (
    customer_id INT AUTO_INCREMENT PRIMARY KEY,
    first_name VARCHAR(255),
    last_name VARCHAR(255),
    email VARCHAR(255),
    address VARCHAR(255),
    phone_number VARCHAR(15)
)
"""

create_products_table = """
CREATE TABLE IF NOT EXISTS Products (
    product_id INT AUTO_INCREMENT PRIMARY KEY,
    product_name VARCHAR(255),
    price DECIMAL(10, 2),
    quantity_available INT
)
"""

create_orders_table = """
CREATE TABLE IF NOT EXISTS Orders (
    order_id INT AUTO_INCREMENT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    status VARCHAR(20),
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
)
"""

create_order_details_table = """
CREATE TABLE IF NOT EXISTS Order_Details (
    order_detail_id INT AUTO_INCREMENT PRIMARY KEY,
    order_id INT,
    product_id INT,
    quantity_ordered INT,
    total_price DECIMAL(10, 2),
    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN KEY (product_id) REFERENCES Products(product_id)
)
"""

# Execute SQL commands to create tables
cursor.execute(create_customers_table)
cursor.execute(create_products_table)
cursor.execute(create_orders_table)
cursor.execute(create_order_details_table)

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Database and tables created successfully.")


In [25]:
# Connect to the MySQL database
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="123",
    database="order_processing"
)
cursor = conn.cursor()

def display_available_products():
    cursor.execute("SELECT * FROM Products")
    products = cursor.fetchall()
    print("Available Products:")
    for product in products:
        product_id, product_name, price, quantity_available = product
        print(f"{product_id}. {product_name} - ${price} ({quantity_available} available)")

def place_order(customer_id):
    display_available_products()
    order = []
    total_cost = 0
    
    while True:
        try:
            product_id = int(input("Enter the product ID (or 0 to finish): "))
            if product_id == 0:
                break
            cursor.execute("SELECT * FROM Products WHERE product_id = %s", (product_id,))
            product = cursor.fetchone()
            if product:
                product_id, product_name, price, quantity_available = product
                quantity_ordered = int(input(f"How many {product_name} do you want to order? "))
                if quantity_ordered > quantity_available:
                    print("Sorry, the product is not available in the requested quantity.")
                else:
                    order.append((product_id, quantity_ordered))
                    total_cost += price * quantity_ordered
            else:
                print("Invalid product ID. Please choose a valid product.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")

    # Insert order and order details into the database
    cursor.execute("INSERT INTO Orders (customer_id, order_date, status) VALUES (%s, NOW(), 'Processing')", (customer_id,))
    order_id = cursor.lastrowid
    for product_id, quantity_ordered in order:
        cursor.execute("INSERT INTO Order_Details (order_id, product_id, quantity_ordered, total_price) VALUES (%s, %s, %s, %s)",
                       (order_id, product_id, quantity_ordered, price * quantity_ordered))
        cursor.execute("UPDATE Products SET quantity_available = quantity_available - %s WHERE product_id = %s",
                       (quantity_ordered, product_id))
    
    # Commit the transaction
    conn.commit()
    
    print(f"Order placed successfully! Total cost: ${total_cost}")

# Example usage
customer_id = 1  # Replace with actual customer ID
place_order(customer_id)



Available Products:


KeyboardInterrupt: Interrupted by user

### 3. You are tasked with developing a Python program that connects to a MySQL database, retrieves data from a table, performs some operations on the data, and updates the database with the modified data. Please write Python code to accomplish this task.
import mysql.connector

Instructions:

1. Assume that you have a MySQL database server running with the following details:

1. Host: localhost

il, Port: 3306

lil. Username: your username

iv. Password: your password

Database Name: your database

vi. Table Name: your_table

vil. The table has the following columns: id (int), name (varchar), quantity (int). 


2. Your Python program should:

i. Connect to the MySQL database.

II. Retrieve all records from the your table table.
iii. Calculate the total quantity of all records retrieved,

Iv. Update the quantity column of each record by doubling its value.

v. Commit the changes to the database. 

vi. Close the database connection.

3. Handle any potential errors that may occur during the database connection and data manipulation, such as connection failures or SQL errors.

4. Provide comments in your code to explain each step.


In [33]:
import mysql.connector

try:
    
    # Database connection parameters
    host = "127.0.0.1"
    username = "root"
    password = "123"
    database_name = "your_database"
    table_name = "your_table"
    cursor = conn.cursor()

    # Create the database if it doesn't exist
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")
    cursor.execute(f"""CREATE TABLE IF NOT EXISTS {table_name} (
                   id INT AUTO_INCREMENT PRIMARY KEY,
                   name VARCHAR(255),
                   quantity INT
                   )""")

    # Connect to the MySQL database
    conn = mysql.connector.connect(
        host=host,
        user=username,
        password=password,
        database=database_name
    )
    
    # Create a cursor to interact with the database
    cursor = conn.cursor()
    
       # Sample data to insert
    sample_data = [
        ("ProductA", 10),
        ("ProductB", 20),
        ("ProductC", 15),
        ("ProductD", 30),
        ("ProductE", 25),
    ]

    # Connect to the MySQL database
    conn = mysql.connector.connect(
        host=host,
        user=username,
        password=password,
        database=database_name
    )

    # Create a cursor to interact with the database
    cursor = conn.cursor()

    # Insert sample data into the table
    for data in sample_data:
        name, quantity = data
        cursor.execute(f"INSERT INTO {table_name} (name, quantity) VALUES (%s, %s)", (name, quantity))

    # Commit the changes to the database
    conn.commit()

    # Close the database connection
    conn.close()

    print("Sample data inserted into the table.")

except mysql.connector.Error as e:
    print(f"Error: {e}")


Sample data inserted into the table.


In [35]:
try:
    import mysql.connector
    
    # Database connection parameters
    host = "127.0.0.1"
    username = "root"
    password = "123"
    database_name = "your_database"
    table_name = "your_table"
    
    conn = mysql.connector.connect(
        host=host,
        user=username,
        password=password,
        database=database_name
    )
    cursor = conn.cursor()
# Create a cursor to interact with the database
    cursor = conn.cursor()
    
    # Retrieve all records from the table
    cursor.execute(f"SELECT * FROM {table_name}")
    records = cursor.fetchall()
    
    # Calculate the total quantity of all records
    total_quantity = sum(record[2] for record in records)
    
    # Double the quantity column of each record and update the database
    for record in records:
        record_id, name, quantity = record
        new_quantity = quantity * 2
        cursor.execute(f"UPDATE {table_name} SET quantity = %s WHERE id = %s", (new_quantity, record_id))
    
    # Commit the changes to the database
    conn.commit()
    
    # Close the database connection
    conn.close()
    
    print("Data retrieval, calculation, and update completed successfully.")
    
except mysql.connector.Error as e:
    print(f"Error: {e}")


Data retrieval, calculation, and update completed successfully.


### 4. You are developing an employee management system for a company. The database should store employee Information, including name, salary, department, and hire date. Managers should be able to view and update employee details.

1. Design the database schema for the employee management system. 

2. Write Python code to connect to the database and retrieve a list of employees in a specific department.

3. Implement a feature to update an employee's salary.



##### 1.Design the database schema for the employee management system.
employee_id (Primary Key)
name (VARCHAR) 
salary (DECIMAL)
department (VARCHAR)
hire_date (DATE)


In [37]:
# 2.Write Python code to connect to the database and retrieve a list of employees in a specific department.
import mysql.connector

try:
   
    host = "127.0.0.1"
    username = "root"
    password = "i123"
    database_name = "employee_management"  # Database name for the employee management system
    table_name = "Employees"  # Table name for employee information

    
    sample_data = [
        ("John Doe", 55000.00, "HR", "2023-01-15"),
        ("Jane Smith", 60000.00, "IT", "2023-02-20"),
        ("Michael Johnson", 52000.00, "Finance", "2023-03-10"),
        ("Emily Davis", 58000.00, "Marketing", "2023-04-05"),
        ("David Wilson", 53000.00, "IT", "2023-05-12")
    ]

   
    conn = mysql.connector.connect(
        host=host,
        user=username,
        password=password,
    )
    
   
    cursor = conn.cursor()

    
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")
    
    
    cursor.execute(f"USE {database_name}")

    
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        employee_id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(255),
        salary DECIMAL(10, 2),
        department VARCHAR(255),
        hire_date DATE
    )
    """

    cursor.execute(create_table_query)
    
    for data in sample_data:
        name, salary, department, hire_date = data
        cursor.execute(f"INSERT INTO {table_name} (name, salary, department, hire_date) VALUES (%s, %s, %s, %s)",
                       (name, salary, department, hire_date))
   
   
    conn.commit()
   
    conn.close()
    
    print(f"Database '{database_name}', table '{table_name}', and sample data inserted successfully.")

except mysql.connector.Error as e:
    print(f"Error: {e}")


Database 'employee_management', table 'Employees', and sample data inserted successfully.


In [41]:
#2. Write Python code to connect to the database and retrieve a list of employees in a specific department.
import mysql.connector

try:
    # Database connection parameters
    host = "127.0.0.1"
    username = "root"
    password = "123"
    database_name = "employee_management"  
    table_name = "Employees" 

   
    conn = mysql.connector.connect(
        host=host,
        user=username,
        password=password,
        database=database_name
    )

    
    cursor = conn.cursor()

    # Specify the department you want to retrieve employees for
    target_department = "HR" 

    # Query employees in the specified department
    cursor.execute("SELECT * FROM Employees WHERE department = %s", (target_department,))
    employees = cursor.fetchall()

    
    for employee in employees:
        print(employee)

   
#3. Implement a feature to update an employee's salary.
    # Specify the employee ID and new salary for the update
    employee_id_to_update = 1  # Replace with the desired employee ID
    new_salary = 60000.00  # Replace with the desired new salary

# Update the employee's salary
    cursor.execute("UPDATE Employees SET salary = %s WHERE employee_id = %s", (new_salary, employee_id_to_update))
    conn.commit()
    print(f"Salary updated for employee ID {employee_id_to_update}.")


except mysql.connector.Error as e:
    print(f"Error: {e}")


(1, 'John Doe', Decimal('55000.00'), 'HR', datetime.date(2023, 1, 15))
Salary updated for employee ID 1.
